<a href="https://colab.research.google.com/github/aidyai/vLM/blob/main/vLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/BAAI-DCAI/Bunny.git

Cloning into 'Bunny'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 247 (delta 38), reused 43 (delta 17), pack-reused 161
Receiving objects: 100% (247/247), 24.51 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
!pip install accelerate

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import warnings

In [ ]:
# disable some warnings
transformers.logging.set_verbosity_error()
transformers.logging.disable_progress_bar()
warnings.filterwarnings('ignore')

# set device
torch.set_default_device('cuda')  # or 'cuda'

In [ ]:
# create model
model = AutoModelForCausalLM.from_pretrained(
    'BAAI/bunny-phi-2-siglip',
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
    'BAAI/bunny-phi-2-siglip',
    trust_remote_code=True)

In [ ]:
# text prompt
prompt = "please give me the html and css code for this UI"
text = f"<image>\n{prompt} ASSISTANT:"
text_chunks = [tokenizer(chunk).input_ids for chunk in text.split('<image>')]
input_ids = torch.tensor(text_chunks[0] + [-200] + text_chunks[1], dtype=torch.long).unsqueeze(0)


In [ ]:
# image, sample images can be found in https://huggingface.co/BAAI/bunny-phi-2-siglip/tree/main/images
image = Image.open("/content/UI DESIGN.jpg")
image_tensor = model.process_images([image], model.config).to(dtype=model.dtype)


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [ ]:
# generate
output_ids = model.generate(
    input_ids,
    images=image_tensor,
    max_new_tokens=100,
    use_cache=True)[0]

print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

I'm sorry, but I cannot provide the HTML and CSS code for this UI as it is not possible to see the exact design and layout of the app. However, I can provide you with some general tips on how to create a visually appealing and user-friendly app interface.


In [ ]:
%cd /content

/content


In [ ]:
!rm -r ./llm-awq

In [ ]:
!git clone https://github.com/mit-han-lab/llm-awq.git

Cloning into 'llm-awq'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 719 (delta 205), reused 219 (delta 158), pack-reused 363
Receiving objects: 100% (719/719), 52.21 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (364/364), done.


In [ ]:
%cd /content/llm-awq

/content/llm-awq


In [ ]:
#!python -m pip install .
!python /content/llm-awq/awq/kernels/setup.py install

In [ ]:
import awq_inference_engine

In [ ]:
import torch
import numpy as np

from PIL import Image
import gc
import requests
from io import BytesIO

from transformers import AutoConfig, AutoTokenizer
from accelerate import load_checkpoint_and_dispatch

from awq.quantize.pre_quant import run_awq, apply_awq
from awq.quantize.quantizer import real_quantize_model_weight

from tinychat.utils.load_quant import load_awq_model
from tinychat.utils.tune import device_warmup, tune_all_wqlinears
from tinychat.utils.prompt_templates import get_prompter, get_stop_token_ids
from tinychat.utils.llava_image_processing import process_images, load_image
from tinychat.models.llava_llama import LlavaLlamaForCausalLM
from tinychat.stream_generators.llava_stream_gen import LlavaStreamGenerator
from tinychat.modules import make_quant_norm, make_quant_attn, make_fused_mlp, make_fused_vision_attn



In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# This demo only support single GPU for now

model_path = "/dataset/llava/llava-v1.5-7b"  # Please change here
quant_path = "../quant_cache/llava-v1.5-7b-w4-g128-awq.pt"  # place to dump quant weights

device = "cuda"

In [ ]:
!pip install autoawq

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


model_path = 'llava-hf/llava-v1.6-mistral-7b-hf'
quant_path = './llava-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True}
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

In [ ]:
!git clone https://github.com/DLCV-BUAA/TinyLLaVABench.git
%cd TinyLLaVABench

Cloning into 'TinyLLaVABench'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 609 (delta 64), reused 55 (delta 27), pack-reused 492
Receiving objects: 100% (609/609), 1.96 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (252/252), done.
/content/TinyLLaVABench


In [ ]:
#!conda create -n tinyllava python=3.10 -y
#!conda activate tinyllava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .

In [ ]:
!git clone https://github.com/zhuyiche/llava-phi.git

#https://github.com/zhuyiche/llava-phi/blob/main/scripts/llava_phi/finetune.sh


Cloning into 'llava-phi'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 172 (delta 50), reused 149 (delta 33), pack-reused 0
Receiving objects: 100% (172/172), 336.11 KiB | 22.41 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
%cd llava-phi

/content/llava-phi


In [ ]:
#conda create -n llava_phi python=3.10 -y
#conda activate llava_phi
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .

In [ ]:
from llava_phi.model.builder import load_pretrained_model
from llava_phi.mm_utils import get_model_name_from_path
from llava_phi.eval.run_llava_phi import eval_model

In [ ]:
model_path = "zxmonent/llava-phi"
prompt = "help me with the css and html for this UI"
image_file = "/content/bacgen.png"

In [ ]:
args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    #"conv_mode": "phi",
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1028
})()

eval_model(args)

In [ ]:
! python /content/llava-phi/llava_phi/serve/cli.py --model_path

In [ ]:
!git clone https://github.com/DLCV-BUAA/TinyLLaVABench.git
%cd TinyLLaVABench


Cloning into 'TinyLLaVABench'...
remote: Enumerating objects: 670, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 670 (delta 99), reused 88 (delta 41), pack-reused 492
Receiving objects: 100% (670/670), 1.98 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/TinyLLaVABench


In [ ]:
#conda create -n tinyllava python=3.10 -y
#conda activate tinyllava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .


In [ ]:
from tinyllava.model.builder import load_pretrained_model
from tinyllava.mm_utils import get_model_name_from_path
from tinyllava.eval.run_tiny_llava import eval_model

In [ ]:
model_path = "bczhou/TinyLLaVA-3.1B"
prompt = "I want you to show me how this UI design will look like as you generate its Html and Css Code?"
image_file = "/content/xui.png"

In [ ]:
args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": "phi",
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1028
})()

eval_model(args)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/796M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Here is an example of how the PopSugar website's UI design might look like when generated with HTML and CSS code:
```html
<!DOCTYPE html>
<html>
<head>
    <title>PopSugar</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <header>
      <h1>PopSugar</h1>
      <nav>
        <ul>
          <li><a href="#celebrity">Celebrity</a></li>
          <li><a href="#fashion">Fashion</a></li>
          <li><a href="#fitness">Fitness</a></li>
          <li><a href="#beauty">Beauty</a></li>
          <li><a href="#family">Family</a></li>
          <li><a href="#culture">Culture</a></li>
          <li><a href="#latin">Latin</a></li>
          <li><a href="#video">Video</a></li>
          <li><a href="#shop">Shop</a></li>
        </ul>
      </nav>
    </header>

    <section id="home">
      <h2>Home</h2>
      <p>Welcome to PopSugar! Here's a sneak peek of what you can expect from our site:</p>
      <p>1. <a href="#menta">Menta</a>: The latest news and trends in pop culture.</

In [ ]:
!git clone https://github.com/PKU-YuanGroup/MoE-LLaVA
%cd MoE-LLaVA

!pip install --upgrade pip  # enable PEP 660 support
!pip install -e ".[train]"
!pip install -e .


Cloning into 'MoE-LLaVA'...
remote: Enumerating objects: 1631, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 1631 (delta 280), reused 145 (delta 142), pack-reused 1291
Receiving objects: 100% (1631/1631), 16.98 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (806/806), done.
/content/MoE-LLaVA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/MoE-LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from PIL import Image
from moellava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from moellava.conversation import conv_templates, SeparatorStyle
from moellava.model.builder import load_pretrained_model
from moellava.utils import disable_torch_init
from moellava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

In [ ]:
def main():
    disable_torch_init()
    image = 'moellava/serve/examples/extreme_ironing.jpg'
    inp = 'What is unusual about this image?'
    model_path = 'LanguageBind/MoE-LLaVA-Phi2-2.7B-4e'  # LanguageBind/MoE-LLaVA-Qwen-1.8B-4e or LanguageBind/MoE-LLaVA-StableLM-1.6B-4e
    device = 'cuda'
    load_4bit, load_8bit = False, False  # FIXME: Deepspeed support 4bit or 8bit?
    model_name = get_model_name_from_path(model_path)
    tokenizer, model, processor, context_len = load_pretrained_model(model_path, None, model_name, load_8bit, load_4bit, device=device)
    image_processor = processor['image']
    conv_mode = "phi"  # qwen or stablelm
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    image_tensor = image_processor.preprocess(Image.open(image).convert('RGB'), return_tensors='pt')['pixel_values'].to(model.device, dtype=torch.float16)

    print(f"{roles[1]}: {inp}")
    inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            use_cache=True,
            stopping_criteria=[stopping_criteria])

    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:], skip_special_tokens=True).strip()
    print(outputs)

if __name__ == '__main__':
    main()